In [1]:
import sys
import pathlib
import numpy as np
import pandas as pd
sys.path.insert(0, "../0.data-download/scripts/")
from data_loader import load_data
from sklearn.model_selection import train_test_split

In [2]:
# load all of the data 
data_directory = "../0.data-download/data/"
dfs = load_data(data_directory, adult_or_pediatric = "all")
dependency_df = dfs[1]
sample_df = dfs[0]

/home/markw/gene_dependency_representations/1.data-exploration/../0.data-download/scripts/data_loader.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  dependency_df = pd.read_csv(dependency_data_file, index_col=0).reset_index().dropna(1)


In [3]:
# verifying that the DepMap_IDs in sample_df and dependency_df are alligned
verify_df = sample_df
verify_df["ID_allignment_verify"] = np.where(dependency_df["DepMap_ID"] == sample_df["DepMap_ID"], "True", "False")
verrify = len(verify_df["ID_allignment_verify"].unique())
print(verify_df["ID_allignment_verify"])
print(f"There is {verrify} output object contained in the ID_allignment_verify column \n")

0       True
1       True
2       True
3       True
4       True
        ... 
1081    True
1082    True
1083    True
1084    True
1085    True
Name: ID_allignment_verify, Length: 1086, dtype: object
There is 1 output object contained in the ID_allignment_verify column 



In [4]:
# assign 'age_categories' and 'sex' columns to the dependency dataframe as a single column
presplit_dependency_df = dependency_df.assign(age_and_sex = sample_df.age_categories.astype(str) + "_" + sample_df.sex.astype(str))
presplit_dependency_df

,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZW10 (9183),ZWILCH (55055),ZWINT (11130),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009),age_and_sex
0,ACH-000001,0.094568,0.012519,0.027460,0.025962,0.073412,0.027340,0.020199,0.284733,0.022084,...,0.555867,0.037449,0.080585,0.004241,0.082956,0.012000,0.003592,0.012679,0.324623,Adult_Female
1,ACH-000004,0.012676,0.049011,0.075933,0.033215,0.013176,0.097497,0.005015,0.153166,0.007358,...,0.007427,0.038768,0.230569,0.011203,0.060266,0.128375,0.005911,0.004645,0.042530,Adult_Male
2,ACH-000005,0.053957,0.027968,0.010139,0.005448,0.018599,0.081636,0.005457,0.159904,0.050884,...,0.071035,0.017479,0.274568,0.033416,0.034712,0.092832,0.012482,0.020843,0.050412,Adult_Male
3,ACH-000007,0.026704,0.083588,0.008853,0.011299,0.027288,0.028349,0.032573,0.166503,0.047045,...,0.213754,0.196233,0.615338,0.005432,0.035241,0.138445,0.103161,0.146222,0.274833,Adult_Male
4,ACH-000009,0.059383,0.051826,0.015370,0.011721,0.030062,0.078373,0.042128,0.184783,0.032335,...,0.169463,0.152385,0.405712,0.056461,0.189550,0.328064,0.035161,0.058402,0.269194,Adult_Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,ACH-002834,0.012720,0.083840,0.013308,0.013315,0.107286,0.079063,0.013340,0.717478,0.040061,...,0.493501,0.027170,0.636352,0.029567,0.029711,0.024132,0.174946,0.091510,0.421153,Missing_Female
1082,ACH-002847,0.058547,0.079576,0.036707,0.030234,0.069976,0.032828,0.012784,0.476383,0.096910,...,0.731151,0.476423,0.967625,0.010776,0.072452,0.131375,0.115486,0.094022,0.438092,Adult_Female
1083,ACH-002874,0.169875,0.017430,0.017965,0.008615,0.117915,0.075693,0.087961,0.144152,0.140377,...,0.462012,0.710398,0.747477,0.017923,0.077515,0.094231,0.166188,0.058117,0.302708,Adult_Female
1084,ACH-002875,0.035519,0.099789,0.071748,0.048479,0.245413,0.154663,0.045155,0.324723,0.089083,...,0.090844,0.329219,0.168367,0.017571,0.101216,0.087739,0.050947,0.182081,0.349872,Adult_Male


In [5]:
groups = sample_df.groupby("age_categories")
df_list = []
for name, df in groups:
    
    # only looking for samples that contain Adult or Pediatric information
    if name == "Adult" or name == "Pediatric":
        df_list.append(df)
        
# merge sample dataframes through concatentation and reorganize so that DepMap_IDs are in alphabetical order
new_df = pd.concat(df_list, axis=0)
new_df.reset_index(drop=True)
new_df = new_df.set_index("DepMap_ID")
new_df = new_df.sort_index(ascending=True)
new_df = new_df.reset_index()

In [6]:
# creating a list of DepMap_IDs that correlate to pediatric and adult samples
PA_dependency_IDs = new_df["DepMap_ID"].tolist()

PA_IDs = set(PA_dependency_IDs) & set(presplit_dependency_df["DepMap_ID"].tolist())

# creating a new gene dependency data frame containing correlating DepMap_IDs to the filtered sample info IDs
PA_dependency_df = presplit_dependency_df.loc[presplit_dependency_df["DepMap_ID"].isin(PA_IDs)].reset_index(drop=True)

In [7]:
#split the data based on age category and sex
train_df, test_df = train_test_split(
    PA_dependency_df, 
    test_size = .15, 
    stratify= PA_dependency_df.age_and_sex
)

In [8]:
# save the TESTING dataframe 
test_df = test_df.reset_index(drop=True)
testing_df_output = pathlib.Path("../0.data-download/data/VAE_test_df.csv")
test_df.to_csv(testing_df_output, index = False)
print(test_df.shape)
test_df.head(3)

(133, 16710)


,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZW10 (9183),ZWILCH (55055),ZWINT (11130),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009),age_and_sex
0,ACH-001651,0.016463,0.008308,0.049038,0.007631,0.141102,0.120387,0.015117,0.178122,0.030336,...,0.046703,0.056306,0.322720,0.013617,0.019262,0.035910,0.031308,0.007752,0.194285,Adult_Female
1,ACH-000431,0.026634,0.051856,0.091913,0.005654,0.050938,0.065029,0.017303,0.109283,0.028732,...,0.100235,0.252447,0.774107,0.015592,0.045275,0.410674,0.014537,0.055549,0.644145,Adult_Male
2,ACH-001028,0.041097,0.094812,0.031027,0.025604,0.055969,0.115373,0.067750,0.134096,0.065525,...,0.103624,0.256184,0.866614,0.039735,0.073950,0.057607,0.018593,0.120312,0.577451,Pediatric_Female


In [9]:
# save the TRAINING dataframe 
train_df = train_df.reset_index(drop=True)
training_df_output = pathlib.Path("../0.data-download/data/VAE_train_df.csv")
train_df.to_csv(training_df_output, index = False)
print(train_df.shape)
train_df.head(3)

(751, 16710)


,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZW10 (9183),ZWILCH (55055),ZWINT (11130),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009),age_and_sex
0,ACH-001607,0.036501,0.046117,0.028447,0.024559,0.177994,0.162455,0.044815,0.404047,0.047244,...,0.204553,0.501764,0.796614,0.031186,0.052418,0.094709,0.105993,0.098644,0.190669,Adult_Female
1,ACH-001652,0.005326,0.085049,0.027489,0.007538,0.049841,0.016074,0.033915,0.266981,0.016402,...,0.598092,0.461756,0.894382,0.060168,0.054313,0.107463,0.019738,0.013748,0.509563,Adult_Female
2,ACH-000037,0.033884,0.007065,0.045215,0.012762,0.089646,0.031148,0.015177,0.657984,0.273457,...,0.020136,0.006008,0.303327,0.072859,0.034732,0.058966,0.032731,0.110653,0.338194,Adult_Female
